In [1]:
from MCpredict import *
import pandas as pd
import ipywidgets as widgets
from collections import defaultdict
import collections
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import HBox, VBox

************************************ EXAMPLE ************************************************

In [2]:
pred_3 = MCpredict(3)
pred_4 = MCpredict(4)

In [3]:
print(pred_3.predict_seq('Wood,Trsgi'))
out_list = pred_4.predict_seq('MarineSediment,Sr/Ca')
print(out_list)

{'0': ['Wood'], '1': ['Trsgi'], '2': ['NA', 'permil PDB', 'g/cm/yr', 'degC', 'cm/yr']}
{'0': ['MarineSediment'], '1': ['Sr/Ca'], '2': ['T', 'NA', 'Temperature And Salinity', 'Salinity', 'D18Osw'], '3': [['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', 'Surface Water'], ['NA', 'Surface Water', 'Bottom Water', 'Air Surface', 'Air Condensationlevel'], ['Sea Surface', 'Surface Water', 'Bottom Water', 'Air Surface', 'Air Condensationlevel'], ['Surface Water', 'Sea Surface', 'Bottom Water', 'Air Surface', 'Air Condensationlevel'], ['Sea Surface', 'Bottom Water', 'Surface Water', 'Surface', 'Subsurface']]}


## DEMO ##

In [43]:
archive_type_dropdown = widgets.Dropdown(options = sorted(list(predict_for_chain_type.names_set[0])), description='Archive Type')

NameError: name 'predict_for_chain_type' is not defined

In [44]:
output_proxy = widgets.Output()
output_units = widgets.Output()
output_int_var = widgets.Output()
output_int_var_det = widgets.Output()

In [45]:
style = {'description_width': 'initial', 'select_width' : 'initial'}

# proxy_obs_type_dropdown = widgets.Combobox(options = [], description='Proxy Observation Type', style=style, ensure_option=True,
#     disabled=False)
# units_dropdown = widgets.Combobox(options = [], description='Units', style=style, ensure_option=True,
#     disabled=False)
# int_var_dropdown = widgets.Combobox(options = [], description='Interpretation Variable', style=style, ensure_option=True,
#     disabled=False)
# int_var_det_dropdown = widgets.Combobox(options = [], description='Interpretation Variable Detail', style=style, ensure_option=True,
#     disabled=False)

proxy_obs_type_dropdown = widgets.Dropdown(options = [], description='Proxy Observation Type', style=style)
units_dropdown = widgets.Dropdown(options = [], description='Units', style=style)
int_var_dropdown = widgets.Dropdown(options = [], description='Interpretation Variable', style=style)
int_var_det_dropdown = widgets.Dropdown(options = [], description='Interpretation Variable Detail', style=style, ensure_option=True,
    disabled=False)

In [46]:
layout_hidden  = widgets.Layout(visibility = 'hidden')
layout_visible = widgets.Layout(visibility = 'visible')

In [47]:
def visible_txt(b):
    combobox.layout = layout_visible

def hidden_txt(b):
    combobox.layout = layout_hidden

In [202]:
def displayChildren(options_list, dropdown_obj, dropdown_eventhandler, to_observe):
#     print('inside display children')
#     print(options_list)
    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')

In [203]:
def int_var_dropdown_eventhandler(change):
    global input_sent
    
    output_int_var_det.clear_output()
#     int_var_det_dropdown.layout = layout_visible
    
    int_var_value = change.new
    input_sent += ',' + int_var_value
    results = predict_for_chain_type.predict_seq(input_sent)

    displayChildren(results['3'], int_var_det_dropdown, None, False)
    with output_int_var_det:
        display(int_var_det_dropdown)

In [204]:
def proxy_obs_type_dropdown_eventhandler(change):
    global input_sent
    
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    
    proxy_obs_type_value = change.new
    input_sent += ',' + proxy_obs_type_value
    results = predict_for_chain_type.predict_seq(input_sent)
    
    if predict_for_chain_type.chain_length == 3:
#         units_dropdown.layout = layout_visible
        displayChildren(results['2'], units_dropdown, None, False)
        with output_units:
            display(units_dropdown)
    else:
#         int_var_dropdown.layout = layout_visible
        displayChildren(results['2'], int_var_dropdown, int_var_dropdown_eventhandler, True)
        with output_int_var:
            display(int_var_dropdown)

In [205]:
def archive_type_dropdown_eventhandler(change):
    global input_sent
    
    # value from archive type dropdown
    archive_type_value = change.new
    input_sent = archive_type_value
    
    # clear existing values from all children drop downs
    output_proxy.clear_output()
    
    output_units.clear_output()
#     units_dropdown.layout = layout_hidden
    
    output_int_var.clear_output()
    output_int_var_det.clear_output()
#     int_var_dropdown.layout = layout_hidden
#     int_var_det_dropdown.layout = layout_hidden
    
    # get prediction results from MC model
    results = predict_for_chain_type.predict_seq(input_sent)
    #print('results', results)
    displayChildren(results['1'], proxy_obs_type_dropdown, proxy_obs_type_dropdown_eventhandler, True)
    
    with output_proxy:
        display(proxy_obs_type_dropdown)
    
# archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

# display(archive_type_dropdown)

In [206]:
def chain_type_dropdown_eventhandler(change):
    global input_sent, predict_for_chain_type
    
    predict_for_chain_type = MCpredict(change.new)
    
    input_sent = ''
    
    output_proxy.clear_output()
    
    output_units.clear_output()
#     units_dropdown.layout = layout_hidden
    
    output_int_var.clear_output()
    output_int_var_det.clear_output()
#     int_var_dropdown.layout = layout_hidden
#     int_var_det_dropdown.layout = layout_hidden
    
    archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')
    display(archive_type_dropdown)

In [207]:
chain_length_dropdown.observe(chain_type_dropdown_eventhandler, names='value')

display(chain_length_dropdown)

Dropdown(description='Chain Type', options=(3, 4), value=3)

Dropdown(description='Archive Type', options=('Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'MarineSediment…

results {'0': ['GlacierIce'], '1': ['D18O', 'Dd', 'Thickness', 'Depth', 'Wood'], '2': [['T', 'Temperature', 'NA', 'D18Osw', 'Temperature And Salinity'], ['T', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['T', 'NA', 'Wood', 'MXD', 'Composite'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core']], '3': [[['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', 'Wood'], ['Sea Surface', 'Thermocline', 'Bottom Water', 'Surface Water', 'Subsurface'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Sea Surface', 'Bottom Water', 'Surface Water', 'Surface', 'Subsurface'], ['Sea Surface', 'Wood', 'MXD', 'LakeSediment', 'Composite']], [['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', 'Wood'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core'], ['T', 'NA', 'Wood', 'MXD', 'Composite'], ['Thickness', 'Depth', 'Core', 'Uk37', 'TEX86'], ['T', 'Wood', 'MXD', 'LakeSediment', 'Composite']], [['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', '

Dropdown(description='Archive Type', index=1, options=('Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'Marin…

results {'0': ['Coral'], '1': ['D18O', 'Calcification', 'Sr/Ca', 'D13C', 'Composite'], '2': [['permil', 'per mil', 'per mil VPDB', 'per mil PDB', 'per mil SMOW'], ['g/cm/yr', 'mm/yr', 'cm/yr', 'Wood', 'MXD'], ['mmol/mol', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['permil', 'per mil', 'Wood', 'MXD', 'Composite'], ['percent', 'Wood', 'MXD', 'LakeSediment', 'Composite']]}
results {'0': ['Speleothem'], '1': ['D18O', 'SE', 'Wood', 'MXD', 'Composite'], '2': [['permil', 'per mil', 'per mil VPDB', 'per mil PDB', 'per mil SMOW'], ['NA', 'degC', 'Wood', 'MXD', 'Composite'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['percent', 'Wood', 'MXD', 'LakeSediment', 'Composite']]}
results {'0': ['Coral'], '1': ['D18O', 'Calcification', 'Sr/Ca', 'D13C', 'Composite'], '2': [['permil', 'per mil', 'per mil VPDB', 'per mil PDB', 'per mil SMOW'], ['g/cm/yr', 'mm/yr', 'cm/yr', 'Wood', 'MXD'], ['mmol/mol', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['permil'

Dropdown(description='Archive Type', options=('Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'MarineSediment…

In [195]:
if predict_for_chain_type.chain_length == 3:
    first_line = output_proxy
    second_line = output_units
    VBox([first_line, second_line])
else:
    first_line = output_proxy
    second_line = output_int_var
    third_line = output_int_var_det
    VBox([first_line, second_line, third_line])

### New implementation of DropDown with text box

In [38]:
predict_chain3 = MCpredict(3)
predict_chain4 = MCpredict(4)
input_sent_list = []

In [39]:
style = {'description_width': 'initial', 'select_width' : 'initial'}

In [40]:
def display_archive_types(archive_types_list):
    archive_types_list.sort()
    archive_types_list.insert(0,'Select')
    return archive_types_list

In [41]:
archive_type_dropdown = widgets.Dropdown(options = display_archive_types(list(predict_chain3.names_set[0])), description='Archive Type')
output_proxy = widgets.Output()
output_units = widgets.Output()
output_int_var = widgets.Output()
output_int_var_det = widgets.Output()

proxy_dropdown = widgets.Combobox(placeholder='Select',options = [], description='proxy observation type'
        , style=style, ensure_option=True, disabled=False)
units_dropdown = widgets.Combobox(placeholder='Select',options =[], description='units(Proxy Observation Type)'
        , style=style, ensure_option=True, disabled=False)
int_var_dropdown = widgets.Combobox(placeholder='Select',options = [], description='interpretation/variable'
        , style=style, ensure_option=True, disabled=False)
int_var_det_dropdown = widgets.Combobox(placeholder='Select',options = [], description='interpretation/variableDetail'
        , style=style, ensure_option=True, disabled=False)


def int_var_dropdown_eventhandler(change):
    global input_sent_list
    output_int_var_det.clear_output()
    int_var_type_value = change.new
    
#     if int_var_type_value == 'Select':
#         print('Please select a value')
#     else:
    input_sent_list = input_sent_list[:2]
    input_sent_list.append(int_var_type_value)
    input_sent = (',').join(input_sent_list)
    res = predict_chain4.predict_seq(input_sent)

    displayChildren(res['3'], int_var_det_dropdown, None, False)

    with output_int_var_det:
        clear_output()
        display(int_var_det_dropdown)

def proxy_dropdown_eventhandler(change):
    global input_sent_list
    
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    
    proxy_obs_type_value = change.new
    
    input_sent_list = input_sent_list[:1]
    input_sent_list.append(proxy_obs_type_value)
    input_sent = (',').join(input_sent_list)
    
    res_chain3 = predict_chain3.predict_seq(input_sent)
    res_chain4 = predict_chain4.predict_seq(input_sent)
    
    displayChildren(res_chain3['2'], units_dropdown, None, False, is_units=True)
    displayChildren(res_chain4['2'], int_var_dropdown, int_var_dropdown_eventhandler, True)
    
    with output_units:
#         clear_output()
        display(units_dropdown)
    with output_int_var:
#         clear_output()
        display(int_var_dropdown)
#     with output_int_var_det:
#         clear_output()
    
def displayChildren(options_list, dropdown_obj, dropdown_eventhandler, to_observe, is_units = False):
    
    if is_units:
        if options_list and options_list[0] == 'NA':
            options_list = ['Unitless']
        else:
            options_list = [x if x != 'NA' else 'Unitless' for x in options_list]   
    else:
        if options_list and options_list[0] == 'NA':
            options_list = ['NA']
#     options_list.insert(0, 'Select')
    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')
    
def archive_type_dropdown_eventhandler(change):
    global input_sent_list
    archive_type_value = change.new
    input_sent_list = []
    input_sent_list.append(archive_type_value)
    
    output_proxy.clear_output()
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    
#     with output_proxy:
#         clear_output()
    
    results = predict_chain3.predict_seq(archive_type_value)
    # print('results - from archive', results)
    displayChildren(results['1'], proxy_dropdown, proxy_dropdown_eventhandler, True)
    
    with output_proxy:
        display(proxy_dropdown)
    
archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

display(archive_type_dropdown)

Dropdown(description='Archive Type', options=('Select', 'Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'Mari…

In [42]:
first_line = HBox([output_proxy])
second_line = HBox([output_units])
third_line = HBox([output_int_var, output_int_var_det])
VBox([first_line, second_line, third_line])

### New implementation with dropdown and text box

In [43]:
predict_chain3 = MCpredict(3)
predict_chain4 = MCpredict(4)
input_sent_list = []

In [44]:
style = {'description_width': 'initial', 'select_width' : 'initial'}

In [45]:
def display_archive_types(archive_types_list):
    archive_types_list.sort()
    archive_types_list.insert(0,'Select')
    return archive_types_list

In [46]:
archive_type_dropdown = widgets.Dropdown(options = display_archive_types(list(predict_chain3.names_set[0])), description='Archive Type')
output_proxy = widgets.Output()
output_units = widgets.Output()
output_int_var = widgets.Output()
output_int_var_det = widgets.Output()

out_text_proxy = widgets.Output()
out_text_units = widgets.Output()
out_text_int_var = widgets.Output()
out_text_int_var_det = widgets.Output()

proxy_dropdown = widgets.Dropdown(placeholder='Select',options = [], description='proxy observation type'
        , style=style, ensure_option=True, disabled=False)
units_dropdown = widgets.Dropdown(placeholder='Select',options =[], description='units(Proxy Observation Type)'
        , style=style, ensure_option=True, disabled=False)
int_var_dropdown = widgets.Dropdown(placeholder='Select',options = [], description='interpretation/variable'
        , style=style, ensure_option=True, disabled=False)
int_var_det_dropdown = widgets.Dropdown(placeholder='Select',options = [], description='interpretation/variableDetail'
        , style=style, ensure_option=True, disabled=False)
proxy_text = widgets.Text(placeholder='Other value', description='other proxy value', disabled=False)
units_text = widgets.Text(placeholder='Other value', description='other units value', disabled=False)
int_var_text = widgets.Text(placeholder='Other value', description='other interpretation/variable value', disabled=False)
int_var_det_text = widgets.Text(placeholder='Other value', description='other interpretation/variableDetail value', disabled=False)


def int_var_dropdown_eventhandler(change):
    global input_sent_list
    output_int_var_det.clear_output()
    int_var_type_value = change.new
    
    if int_var_type_value == 'Select':
        print('Please select a value')
    else:
        input_sent_list = input_sent_list[:2]
        input_sent_list.append(int_var_type_value)
        input_sent = (',').join(input_sent_list)
        res = predict_chain4.predict_seq(input_sent)

        displayChildren(res['3'], int_var_det_dropdown, None, False)

    with output_int_var_det:
        clear_output()
        display(int_var_det_dropdown)

def proxy_dropdown_eventhandler(change):
    global input_sent_list
    
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    
    proxy_obs_type_value = change.new
    
    input_sent_list = input_sent_list[:1]
    input_sent_list.append(proxy_obs_type_value)
    input_sent = (',').join(input_sent_list)
    
    res_chain3 = predict_chain3.predict_seq(input_sent)
    res_chain4 = predict_chain4.predict_seq(input_sent)
    
    displayChildren(res_chain3['2'], units_dropdown, None, False, is_units=True)
    displayChildren(res_chain4['2'], int_var_dropdown, int_var_dropdown_eventhandler, True)
    
    with output_units:
        clear_output()
        display(units_dropdown)
    with output_int_var:
        clear_output()
        display(int_var_dropdown)
    with output_int_var_det:
        clear_output()
    
def displayChildren(options_list, dropdown_obj, dropdown_eventhandler, to_observe, is_units = False):
    
    if is_units:
        if options_list and options_list[0] == 'NA':
            options_list = ['Unitless']
        else:
            options_list = [x if x != 'NA' else 'Unitless' for x in options_list]   
    else:
        if options_list and options_list[0] == 'NA':
            options_list = ['NA']
    options_list.insert(0, 'Select')
    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')
    
def archive_type_dropdown_eventhandler(change):
    global input_sent_list
    archive_type_value = change.new
    input_sent_list = []
    input_sent_list.append(archive_type_value)
    
    output_proxy.clear_output()
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    
#     out_text_proxy.clear_output()
#     out_text_units.clear_output()
#     out_text_int_var.clear_output()
#     out_text_int_var_det.clear_output()
    
    with output_proxy:
        clear_output()
    
    results = predict_chain3.predict_seq(archive_type_value)
    # print('results - from archive', results)
    displayChildren(results['1'], proxy_dropdown, proxy_dropdown_eventhandler, True)
    
    with output_proxy:
        display(proxy_dropdown)
    
archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

display(archive_type_dropdown)

Dropdown(description='Archive Type', options=('Select', 'Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'Mari…

In [47]:
first_line = HBox([output_proxy])
second_line = HBox([output_units])
third_line = HBox([output_int_var, output_int_var_det])
VBox([first_line, second_line, third_line])